In [1]:
import os
import mahotas as mh
import numpy as np
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.html.widgets import interact, fixed
import pandas as pd
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # 10 x 8 inches
plt.gray()

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [2]:
import warnings
warnings.filterwarnings('ignore')

Import a classifier

In [4]:
from sklearn.externals import joblib
clf = joblib.load('/cellar/users/ramarty/Data/ants/gold_standard/classifiers/random_forest.size_100.pkl')

In [6]:
files = list(set([x.split('.')[0] for x in os.listdir('/cellar/users/ramarty/Data/ants/gold_standard/photos/') if ('box101' in x) or ('box61' in x)]))

In [7]:
# import test files
test_file = '/cellar/users/ramarty/Data/ants/gold_standard/photos/{0}.pgm'.format(files[19])
test_file_gs = '/cellar/users/ramarty/Data/ants/gold_standard/photos/{0}.png'.format(files[19])

In [8]:
training_size = 100

Simple training to create a prediction matrix

In [107]:
def test_photo(test_file, training_size):
    prediction_matrix = []
    im = mh.imread(test_file)
    y1, y2, x1, x2 = 0, im.shape[0], 0, im.shape[1]
    print y1, y2, x1, x2
    # cycle through photo
    y_total, x_total = im[y1:y2, x1:x2].shape
    print y_total, x_total
    # assigning values
    for i in range(0, y_total-training_size, training_size):
        prediction_array = [[] for x in range(training_size)]
        for j in range(0, x_total-training_size, training_size):
            test_features = mh.features.haralick(im[y1+i:y1+i+training_size, x1+j:x1+j+training_size], return_mean_ptp=True) 
            prediction = clf.predict_proba(test_features)[0][0]
            for k in range(training_size):
                prediction_array[k].extend([prediction for w in range(training_size)])
        for k in range(training_size):
            prediction_matrix.append(prediction_array[k])
    # filling in edge gaps
    arrays_needed = im.shape[0] - len(prediction_matrix)
    values_needed = im.shape[1] - len(prediction_matrix[0])
    for i, prediction_array in enumerate(prediction_matrix):
        prediction_matrix[i].extend([0 for x in range(values_needed)])
    for i in range(arrays_needed):
        prediction_matrix.append([0 for x in range(im.shape[1])])
    return prediction_matrix

In [108]:
import warnings
warnings.filterwarnings('ignore')

In [109]:
prediction_matrix = test_photo(test_file, training_size)

0 4560 0 3048
4560 3048


In [111]:
# convert to numpy matrix
prediction_matrix_np = np.matrix(prediction_matrix)

In [113]:
pickle.dump( prediction_matrix_np, open( "/cellar/users/ramarty/Data/ants/gold_standard/predictions/test_matrix.p", "wb" ) )

Comparison between prediction_matrix and gs

In [114]:
prediction_matrix = pickle.load( open( "/cellar/users/ramarty/Data/ants/gold_standard/predictions/test_matrix.p", "rb" ) )

In [115]:
prediction_matrix.shape

(4560, 3048)

In [77]:
gs = mh.imread(test_file_gs)

In [78]:
# convert into 0s and 1s
gs.shape

(4560, 3048, 3)

In [79]:
def f(x):
    if (x[0] == 246) & (x[1] == 255) & (x[2] == 0):
        return 1
    else:
        return 0


In [81]:
gs_binary = np.arange(len(gs)*len(gs[0])).reshape(len(gs), len(gs[0]))
for row in range(len(gs)):
    for col in range(len(gs[0])):
        gs_binary[row][col] = f(gs[row][col])

In [82]:
gs_binary.shape

(4560, 3048)

In [89]:
gs_binary

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Find difference in matrices?

In [116]:
diff = gs_binary - prediction_matrix

In [122]:
diff_abs = abs(diff)

In [133]:
diff[np.where( diff > 0 )]

matrix([[ 0.2,  0.2,  0.2, ...,  1. ,  1. ,  1. ]])

In [151]:
total_dim = gs_binary.shape[0] * gs_binary.shape[1]

In [144]:
diff[(diff > 0)].sum() / gs_binary.sum()

0.51320518925742564

In [154]:
abs(diff[(diff < 0)].sum()) / (total_dim - gs_binary.sum())

0.029834144937496147

In [148]:
gs_binary[(gs_binary < 1)].sum()

0

In [152]:
total_dim - gs_binary.sum()

13189106

In [ ]:
# subtract? absolute value? sum? Or difference of some kind?